# Final Project #

## Create the DataFrame ##

In [ ]:
import pandas as pd
import numpy as mp

In [ ]:
# Create a reference to the CSV and import it into a Pandas DataFrame
csv_path = "Resources/student-por.csv"
students_df = pd.read_csv(csv_path)
students_df.head()

In [ ]:
students_df.describe()

In [ ]:
students_df.columns

In [ ]:
# need to pull out Walc before get dummies on dataset - need Walc later to bin
pulled_Walc = students_df[['Walc']].copy()
pulled_Walc

In [ ]:
# Drop unrelated columns and Walc
students_df = students_df.drop(['school','G1', 'G2', 'G3', 'Dalc', 'Walc'], axis=1)
students_df.head()

In [ ]:
students_df.dtypes

In [ ]:
students_df.head()

In [ ]:
# perform get dummies on data set (less Walc) to binary encode for scaling
encoded_students_df = pd.get_dummies(students_df)
encoded_students_df.head()

In [ ]:
# keep encoded df clean from y data for prediction purposes
working_students_df = encoded_students_df.copy()
working_students_df.head()

In [ ]:
working_students_df.dtypes

In [ ]:
pulled_Walc.dtypes

In [ ]:
# insert Walc back into encoded dataframe
working_students_df['Walc']=pulled_Walc['Walc']
working_students_df

In [ ]:
# Create the bins in which Drinking levels will be held
bins = [0,1,2,3,4,5]

# Create the names for the five bins
group_names = ["Not at all", "Slightly", "Moderately", "Very", "Extremely"]

In [ ]:
# Create column to contain new bins
working_students_df["Likelyhood to Drink"] = pd.cut(working_students_df["Walc"], bins, labels=group_names)
working_students_df = working_students_df.drop(['Walc'], axis=1)

working_students_df

In [ ]:
# Reformat data (x-inputs, y labels)
data = working_students_df.values
X = data[:, 0:52]
y = data[:, 52]

y = y.reshape(-1, 1)
print(X)
print(y)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)



## Data Preprocessing ##

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)
print(encoded_y_train)
print(encoded_y_test)

In [ ]:
for label, original_class in zip(encoded_y_train, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

In [ ]:
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Create Deep Learning Model ##

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [ ]:
from tensorflow.keras.layers import Dense
number_inputs = 52
number_hidden_nodes = 150
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [ ]:
# add another layer??
number_classes = 5
model.add(Dense(units=number_classes, activation='relu'))
# model.add(Dense(units=number_classes, activation='relu'))
# model.add(Dense(units=number_classes, activation='relu'))
# model.add(Dense(units=number_classes, activation='relu'))
# model.add(Dense(units=number_classes, activation='relu'))
# model.add(Dense(units=number_classes, activation='relu'))

In [ ]:
# add another layer??
number_classes = 5
model.add(Dense(units=number_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train_scaled.shape=(486,52)

In [ ]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)

## Quantify Trained Model ##

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

## Save Trained Model ##

In [ ]:
# Save the model
model.save("students_deep_learning_trained.h5")

In [ ]:
import numpy as np
predict_student = encoded_students_df.iloc[4]

predict_student = np.expand_dims(predict, axis=0)

model.predict_classes(predict_student)
predict